<a href="https://colab.research.google.com/github/jaiwon880/MachineLearning_jw/blob/main/web/%EC%95%84%ED%8C%8C%ED%8A%B8_%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_2%EC%9B%94_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 아파트 실거래가 예측

In [1]:
# INPUT : 도/면적/계약년월/건축년도/유형
# OUTPUT : 금액
# EX) 서울 /50 / 2020 / 직거래 → 100억?
# EX2) 경기도/ 30 / 1990/ 중개 → ???억

# 시군구 전처리 - ex) 경기도 OO군 OO시 → 경기도 df[’’].split(’ ‘)[0]
# 거래유형 전처리 - 중개 : 0 직 : 1
# 피쳐엔지니어링 - ex) 면적당 금액?

In [ ]:
#@title 폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
plt.rc('font', family='NanumBarunGothic') 

In [16]:
#@title 시군구 - 전용면적, 계약년월, 거래 금액, 건축년도, 거래유형
df_apt2 = pd.read_csv('https://github.com/jaiwon880/ML_Projiect/raw/main/Data/%EC%95%84%ED%8C%8C%ED%8A%B8_%EB%A7%A4%EB%A7%A4__%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_20230321134120_2%EC%9B%94.csv')
df_apt2

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
0,강원도 강릉시 견소동,84.9450,202302,7,"25,500",1997.0,중개거래
1,강원도 강릉시 견소동,84.9450,202302,23,"16,000",1997.0,직거래
2,강원도 강릉시 견소동,84.9450,202302,23,"23,000",1997.0,직거래
3,강원도 강릉시 견소동,84.9900,202302,11,"29,000",2005.0,중개거래
4,강원도 강릉시 교동,164.0098,202302,18,"59,500",2009.0,중개거래
...,...,...,...,...,...,...,...
32006,충청북도 충주시 호암동,84.9289,202302,16,"23,000",2007.0,중개거래
32007,충청북도 충주시 호암동,84.9289,202302,21,"23,000",2007.0,중개거래
32008,충청북도 충주시 호암동,84.6800,202302,2,"20,800",2002.0,중개거래
32009,충청북도 충주시 호암동,84.9500,202302,2,"40,000",2019.0,중개거래


In [6]:
df_apt2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32011 entries, 0 to 32010
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       32011 non-null  object 
 1   전용면적(㎡)   32011 non-null  float64
 2   계약년월      32011 non-null  int64  
 3   계약일       32011 non-null  int64  
 4   거래금액(만원)  32011 non-null  object 
 5   건축년도      32005 non-null  float64
 6   거래유형      32011 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 1.7+ MB


In [17]:
df_apt2['거래금액(만원)'].str.strip().replace(' ','')
df_apt2

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
0,강원도 강릉시 견소동,84.9450,202302,7,"25,500",1997.0,중개거래
1,강원도 강릉시 견소동,84.9450,202302,23,"16,000",1997.0,직거래
2,강원도 강릉시 견소동,84.9450,202302,23,"23,000",1997.0,직거래
3,강원도 강릉시 견소동,84.9900,202302,11,"29,000",2005.0,중개거래
4,강원도 강릉시 교동,164.0098,202302,18,"59,500",2009.0,중개거래
...,...,...,...,...,...,...,...
32006,충청북도 충주시 호암동,84.9289,202302,16,"23,000",2007.0,중개거래
32007,충청북도 충주시 호암동,84.9289,202302,21,"23,000",2007.0,중개거래
32008,충청북도 충주시 호암동,84.6800,202302,2,"20,800",2002.0,중개거래
32009,충청북도 충주시 호암동,84.9500,202302,2,"40,000",2019.0,중개거래


In [ ]:
df_apt2.describe(include='all') # 54억 ^_^... 700마넌

In [18]:
df_apt2['거래금액(만원)'].str.strip().replace(' ','')
df_apt2

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
0,강원도 강릉시 견소동,84.9450,202302,7,"25,500",1997.0,중개거래
1,강원도 강릉시 견소동,84.9450,202302,23,"16,000",1997.0,직거래
2,강원도 강릉시 견소동,84.9450,202302,23,"23,000",1997.0,직거래
3,강원도 강릉시 견소동,84.9900,202302,11,"29,000",2005.0,중개거래
4,강원도 강릉시 교동,164.0098,202302,18,"59,500",2009.0,중개거래
...,...,...,...,...,...,...,...
32006,충청북도 충주시 호암동,84.9289,202302,16,"23,000",2007.0,중개거래
32007,충청북도 충주시 호암동,84.9289,202302,21,"23,000",2007.0,중개거래
32008,충청북도 충주시 호암동,84.6800,202302,2,"20,800",2002.0,중개거래
32009,충청북도 충주시 호암동,84.9500,202302,2,"40,000",2019.0,중개거래


In [19]:
#@title 평당가 금액
df_apt2["거래금액(만원)"] = df_apt2["거래금액(만원)"].str.replace(",", "").astype(int) # 거래금액 컬럼의 쉼표를 제거하고 정수형으로 변환
df_apt2["평당가"] = (df_apt2["거래금액(만원)"] / df_apt2["전용면적(㎡)"]) * 3.3 # 평당가 계산하여 새로운 컬럼 추가
df_apt2["평당가"]

0         990.641003
1         621.578669
2         893.519336
3        1126.014825
4        1197.184558
            ...     
32006     893.688721
32007     893.688721
32008     810.581011
32009    1553.855209
32010    1620.240481
Name: 평당가, Length: 32011, dtype: float64

In [20]:
df_apt2 = df_apt2.drop('시군구', axis=1)
df_apt2['거래유형'] = df_apt2['거래유형'].replace({'중개거래': 0, '직거래': 1})

In [22]:
#@title 결측치 건축년도 4개 삭제 
df_apt2.isna().sum()
df_apt2.dropna(inplace=True)

In [21]:
df_apt2.isna().sum()

전용면적(㎡)     0
계약년월        0
계약일         0
거래금액(만원)    0
건축년도        6
거래유형        0
평당가         0
dtype: int64

In [23]:
df_apt2['금리'] = df_apt2.apply(lambda x: 3.25 if ((x['계약년월']== 202301) & (x['계약일'] < 13)) else 3.5, axis=1)

In [24]:
#@title 훈련셋 테스트셋 나누기

X = df_apt2.drop("거래금액(만원)", axis=1)
y = df_apt2[['거래금액(만원)']]


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=100)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(25604, 7) (6401, 7) (25604, 1) (6401, 1)


In [25]:
#@title  선형회귀?

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
def rmse_error(y, y_pred):
  rmse = mean_squared_error(y, y_pred) ** 0.5
  return rmse

print(rmse_error(lr.predict(X_train),y_train))
print(rmse_error(lr.predict(X_test),y_test))

9696.938413015156
8671.654516716419


In [ ]:
# import requests
# from bs4 import BeautifulSoup

# # 크롤링할 페이지 URL
# url = "https://www.google.com/search?q=2022%EB%85%84%2C+2023%EB%85%84+%EA%B8%B0%EC%A4%80%EA%B8%88%EB%A6%AC&oq=2022%EB%85%84%2C+2023%EB%85%84+%EA%B8%B0%EC%A4%80%EA%B8%88%EB%A6%AC&aqs=chrome..69i57j0i512l4j0i30j0i8i30l4.6905j1j7&sourceid=chrome&ie=UTF-8"

# # requests 라이브러리를 사용하여 HTML 코드 가져오기
# response = requests.get(url)
# html = response.text

# # BeautifulSoup 라이브러리를 사용하여 HTML 파싱
# soup = BeautifulSoup(html, "html.parser")

# # 최신 데이터 출력하기
# table = soup.find_all("table", {"class": "ztXv9"})

# for tr in table[0].find_all("tr"):
#     td_list = tr.find_all("td")
#     if len(td_list) == 5:
#         print(td_list[0].text.strip(), td_list[2].text.strip())

In [ ]:
#@title get_dummies 쓰면. . . 이따구로 나와요 화가 나
# df_apt1_2 = pd.get_dummies(df_apt1, columns=df_apt1.describe(include='O').columns)
# df_apt1_2

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=100)